<a href="https://colab.research.google.com/github/antagonisuto/deepLearning/blob/master/A6_MadinaS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
  def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)

        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)


  def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 9216)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

    for epoch in range(1, 10 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

    torch.save(model.state_dict(),"mnist_cnn.pt")
        
if __name__ == '__main__':
    main()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.282588
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.233140
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.954722
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.388927
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.990269
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.564864
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.422303
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.652402
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.336613
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.368194
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.466798
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.442730
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.325608
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.411447
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.626840
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.417758
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.485244
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.389382
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.343537
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.284094
Train Epoch: 1 [12800/60000 (

KeyboardInterrupt: ignored